In [1]:
from GML import sweetviz
from GML import AutoML

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score, fbeta_score
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.metrics import classification_report
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


GML is up to date 
 


Feel free to contact us at: https://github.com/Muhammad4hmed/GML
Your GML is ready!


In [2]:
X_train = pd.read_csv('../Data/X_train_SMOTE.csv') 
y_train = pd.read_csv('../Data/y_train_SMOTE.csv')
X_test = pd.read_csv('../Data/X_test.csv')
y_test = pd.read_csv('../Data/y_test.csv')

contest_test = pd.read_csv('../Data/test_data.csv')
test = pd.read_csv('../Data/test.csv')

In [43]:
X_train.head()

,sex_0,sex_1,任職前工作平均年數,出差數A,出差數B,出差集中度,升遷速度,婚姻狀況_1,婚姻狀況_2,婚姻狀況_3,...,職等_7,職等_8,訓練時數A,訓練時數B,訓練時數C,近一年請假數A,近一年請假數B,近三月請假數A,近三月請假數B,通勤成本
0,0.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,5.0,0.0,2.0,0.0,8.0
1,0.0,1.0,0.0,4.0,0.0,6.0,4.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,2.0,6.0,0.0,4.0,0.0,8.0
2,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,8.0
3,0.0,1.0,0.0,3.0,0.0,1.0,2.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,6.0,0.0,4.0,0.0,9.0
4,1.0,0.0,2.0,1.0,0.0,2.0,2.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,4.0,0.0,4.0,0.0,8.0


In [8]:
X_test.head()

,sex_0,sex_1,任職前工作平均年數,出差數A,出差數B,出差集中度,升遷速度,婚姻狀況_1,婚姻狀況_2,婚姻狀況_3,...,職等_7,職等_8,訓練時數A,訓練時數B,訓練時數C,近一年請假數A,近一年請假數B,近三月請假數A,近三月請假數B,通勤成本
0,0.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,6.0,0.0,3.0,0.0,8.0
1,0.0,1.0,1.0,1.0,0.0,1.0,2.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,6.0,0.0,6.0,0.0,6.0
2,1.0,0.0,1.0,1.0,0.0,0.0,3.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,2.0,6.0,0.0,5.0,0.0,8.0
3,0.0,1.0,0.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,6.0,0.0,3.0,0.0,8.0
4,0.0,1.0,0.0,1.0,0.0,2.0,3.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,6.0,0.0,3.0,0.0,10.0


In [48]:
corr = pd.concat([X_train, y_train], axis=1).corr()['PerStatus'].dropna()
corr_std = np.std(col.values.tolist()[:-1])
corr_value = pd.concat([X_train, y_train], axis=1).corr()['PerStatus'].fillna(0).values

In [51]:
corr_idx = []

for idx, value in enumerate(corr_value[:-1]):
    if value > corr_std * 2:
        corr_idx.append(idx)
        
corr_idx

[1, 8, 12, 29, 50]

# Scale Data between 0 and 1

In [38]:
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scale = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_train.columns)
test_scale = pd.DataFrame(scaler.transform(contest_test.iloc[:, 1:]), columns=X_train.columns)

# Feature Selection

In [145]:
clf = LogisticRegression(random_state=0)

rfe = RFE(clf, 20)
rfe = rfe.fit(X_train_scale, y_train.values.ravel())

In [146]:
important_feature = []

for i, value in enumerate(rfe.ranking_):
    if value == 1:
        important_feature.append(i)
print(len(important_feature))

20


# Feature Selection

In [195]:
X_train_scale

,sex_0,sex_1,任職前工作平均年數,出差數A,出差數B,出差集中度,升遷速度,婚姻狀況_1,婚姻狀況_2,婚姻狀況_3,...,職等_7,職等_8,訓練時數A,訓練時數B,訓練時數C,近一年請假數A,近一年請假數B,近三月請假數A,近三月請假數B,通勤成本
0,0.0,1.0,0.142857,0.100000,0.000000,0.000000,0.111111,1.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.133333,0.363636,0.000000,0.133333,0.000000,0.333333
1,0.0,1.0,0.000000,0.400000,0.000000,0.428571,0.333333,1.0,0.0,0.0,...,1.0,0.0,0.066667,0.0,0.133333,0.454545,0.000000,0.266667,0.000000,0.333333
2,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.111111,1.0,0.0,0.0,...,1.0,0.0,0.000000,0.0,0.000000,0.363636,0.000000,0.000000,0.000000,0.333333
3,0.0,1.0,0.000000,0.300000,0.000000,0.071429,0.111111,1.0,0.0,0.0,...,0.0,0.0,0.066667,0.0,0.066667,0.454545,0.000000,0.266667,0.000000,0.416667
4,1.0,0.0,0.285714,0.100000,0.000000,0.142857,0.111111,1.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.133333,0.272727,0.000000,0.266667,0.000000,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21627,0.0,1.0,0.142857,0.100000,0.000000,0.021754,0.461364,1.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.056515,0.531611,0.000000,0.123182,0.000000,0.378598
21628,1.0,0.0,0.155579,0.100000,0.091094,0.000000,0.019790,1.0,0.0,0.0,...,1.0,0.0,0.000000,0.0,0.127396,0.561647,0.227736,0.466667,0.113868,0.416667
21629,0.0,1.0,0.000000,0.000000,0.000000,0.000000,0.088305,0.0,1.0,0.0,...,1.0,0.0,0.000000,0.0,0.119650,0.473205,0.000000,0.133333,0.000000,0.767105
21630,0.0,1.0,0.000000,0.024817,0.000000,0.000000,0.083536,0.0,1.0,0.0,...,1.0,0.0,0.033090,0.0,0.133333,0.522893,0.000000,0.233090,0.000000,0.333333


## ExtraTree

In [53]:
clf_ExtraTree = ExtraTreesClassifier(max_depth=10, n_jobs=-1)
clf_ExtraTree.fit(X_train_scale, y_train['PerStatus'])

ExtraTreesClassifier(max_depth=10, n_jobs=-1)

In [54]:
importance = clf_ExtraTree.feature_importances_

In [55]:
important_idx = []
std = np.std(importance) * 2

for i,value in enumerate(importance):
    if value > std:
        important_idx.append(i)

In [56]:
important_idx

[0, 1, 6, 7, 8, 12, 13, 29, 32, 33, 34, 36, 50, 202, 219]

## TSNE

In [211]:
X_embedded = TSNE(n_components=2, n_jobs=-1, random_state=40).fit_transform(X_train_scale)

In [213]:
X_test_embedded = TSNE(n_components=2, n_jobs=-1, random_state=40).fit_transform(X_test_scale)

## LDA

In [225]:
lda = LinearDiscriminantAnalysis()
lda.fit(X_train_scale.iloc[:, important_idx], y_train)
X_train_LDA = lda.transform(X_train_scale.iloc[:, important_idx])
X_test_LDA = lda.transform(X_test_scale.iloc[:, important_idx])

# Model Training

## Correlation Model

In [59]:
clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train_scale.iloc[:, corr_idx], y_train)

y_test['Pred'] = clf.predict(X_test_scale.iloc[:, corr_idx])
print('Accuracy of ExtraTreesClassifier on test set: {:.2f}'.format(accuracy_score(y_test['PerStatus'], y_test['Pred'])))

Accuracy of ExtraTreesClassifier on test set: 0.69


In [60]:
target_names = ['0', '1']
print(classification_report(y_test['PerStatus'], y_test['Pred'], target_names=target_names), '\n')
print('fbeta_score: ', fbeta_score(y_test['PerStatus'], y_test['Pred'], average='macro', beta=1.5))

              precision    recall  f1-score   support

           0       0.96      0.70      0.81      2710
           1       0.08      0.48      0.14       154

    accuracy                           0.69      2864
   macro avg       0.52      0.59      0.48      2864
weighted avg       0.91      0.69      0.77      2864
 

fbeta_score:  0.47879386639117194


## Random Forest important Feature Model

In [79]:
clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train_scale.iloc[:, important_idx], y_train)

y_test['Pred'] = clf.predict(X_test_scale.iloc[:, important_idx])
y_train['Pred'] = clf.predict(X_train_scale.iloc[:, important_idx])
print('Accuracy of ExtraTreesClassifier on traub set: {:.2f}'.format(accuracy_score(y_train['PerStatus'], y_train['Pred'])))

Accuracy of ExtraTreesClassifier on traub set: 0.80


In [62]:
target_names = ['0', '1']
print(classification_report(y_test['PerStatus'], y_test['Pred'], target_names=target_names), '\n')
print('fbeta_score: ', fbeta_score(y_test['PerStatus'], y_test['Pred'], average='macro', beta=1.5))

              precision    recall  f1-score   support

           0       0.96      0.78      0.86      2710
           1       0.11      0.50      0.18       154

    accuracy                           0.76      2864
   macro avg       0.54      0.64      0.52      2864
weighted avg       0.92      0.76      0.82      2864
 

fbeta_score:  0.5337445942638522


## TSNE Model

In [214]:
# TSNE Model
clf = MLPClassifier(random_state=1, max_iter=300).fit(X_embedded, y_train)

y_test['Pred'] = clf.predict(X_test_embedded)
print('Accuracy of ExtraTreesClassifier on test set: {:.2f}'.format(accuracy_score(y_test['PerStatus'], y_test['Pred'])))

Accuracy of ExtraTreesClassifier on test set: 0.58


In [215]:
target_names = ['0', '1']
print(classification_report(y_test['PerStatus'], y_test['Pred'], target_names=target_names), '\n')
print('fbeta_score: ', fbeta_score(y_test['PerStatus'], y_test['Pred'], average='macro', beta=1.5))

              precision    recall  f1-score   support

           0       0.93      0.60      0.73      2710
           1       0.04      0.27      0.06       154

    accuracy                           0.58      2864
   macro avg       0.49      0.43      0.40      2864
weighted avg       0.89      0.58      0.69      2864
 

fbeta_score:  0.38010782139005234


## LDA Model

In [226]:
# LDA Model
clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train_LDA, y_train)

y_test['Pred'] = clf.predict(X_test_LDA)
print('Accuracy of ExtraTreesClassifier on test set: {:.2f}'.format(accuracy_score(y_test['PerStatus'], y_test['Pred'])))

Accuracy of ExtraTreesClassifier on test set: 0.65


In [227]:
target_names = ['0', '1']
print(classification_report(y_test['PerStatus'], y_test['Pred'], target_names=target_names), '\n')
print('fbeta_score: ', fbeta_score(y_test['PerStatus'], y_test['Pred'], average='macro', beta=1.5))

              precision    recall  f1-score   support

           0       0.97      0.65      0.78      2710
           1       0.10      0.64      0.17       154

    accuracy                           0.65      2864
   macro avg       0.53      0.65      0.47      2864
weighted avg       0.92      0.65      0.75      2864
 

fbeta_score:  0.4803780777237942


# Submission

In [64]:
contest_y_pred = clf.predict(test_scale.iloc[:, important_idx])

In [65]:
contest_pred_df = pd.DataFrame(contest_y_pred, columns=['Pred'])

In [66]:
contest_pred_df['index'] = contest_test['index']

In [67]:
contest_pred_df

,Pred,index
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4
...,...,...
3716,0,3734
3717,0,3735
3718,1,3736
3719,0,3737


In [78]:
sub = test.reset_index().merge(contest_pred_df, on='index', how='left')[['PerNo', 'Pred']]
sub.columns.values[1] = 'PerStatus'
sub.to_csv('../Data/Sub.csv', index=0)

,PerNo,PerStatus
0,1,0.0
1,3,0.0
2,7,0.0
3,15,0.0
4,16,0.0
...,...,...
3734,8761,0.0
3735,8765,0.0
3736,8767,1.0
3737,8774,0.0
